In [87]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import xgboost as xgb
import math
import csv
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

In [88]:
train_a = pd.read_parquet('../../A/train_targets.parquet')
train_b = pd.read_parquet('../../B/train_targets.parquet')
train_c = pd.read_parquet('../../C/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('../../A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../../B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../../C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../../A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../../B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../../C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../../A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../../B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../../C/X_test_estimated.parquet')

In [89]:
X_train_observed_a['date_forecast'] = pd.to_datetime(X_train_observed_a['date_forecast'])
X_train_observed_a.set_index('date_forecast', inplace=True)

In [90]:
df = X_train_observed_a.copy()

df.index = pd.to_datetime(df.index)

grouped = df.groupby(df.index.date)

def time_difference_from_max_solar_elevation(group):
    time_of_max_solar_elevation = group['sun_elevation:d'].idxmax()
    time_difference = (group.index - time_of_max_solar_elevation).seconds / 3600
    time_difference = time_difference - (time_difference > 12) * 24  # Adjust for differences more than 12 hours
    return pd.Series(time_difference, index=group.index)

# Apply the function to each group
time_difference = grouped.apply(time_difference_from_max_solar_elevation)

# Flatten the resulting multi-index
time_difference = time_difference.reset_index(level=0, drop=True)


In [91]:
time_difference = time_difference.to_frame()

How should i pick the models i want to use?

Is there any point in having specialized models?

The based to collect the models
 - How should i weight them?
    - Not according to MAE on Kaggle?

 Should i fit the dataset only on a subset of the data?

How bad can the worst model in the ensemble be?

Any point in bagging/stacking?

Benefits of regularized greedy forest?

Trasformer architecture?

Is there such a thing as too much tuning?

In [97]:
time_difference.head()

,date_forecast
date_forecast,
2019-06-02 22:00:00,0.00
2019-06-02 22:15:00,0.25
2019-06-02 22:30:00,0.50
2019-06-02 22:45:00,0.75
2019-06-02 23:00:00,1.00
